## MCMC 的秘诀

---

#### 介绍

PyMC 提供了贝叶斯推断的一个强大后台，它能够很好的向用户隐藏 MCMC 等算法的内部实现机制。尽管如此，为了确保估计不会因为 MCMC 算法天然的迭代性而导致偏差，我们还需要注意一些细节，下面的实验中，我们会详细的阐述，这些我们需要注意的细节。

#### 知识点

- 子相关性
- 稀释
- MCMC的初始值的选取
- 先验的选择

---

在进行实验前，我们还是需要加载 PyMC 库，这里请直接运行下面几段代码（具体解释，已在第二个实验中给出）：

In [ ]:
!pip install pymc3==3.8

In [ ]:
#卸载当前环境的 numpy，防止与新版本冲突
!pip  uninstall -y numpy
#更新 numpy
!pip install -U numpy 

In [ ]:
#重启内核
import os
os._exit(00)

In [ ]:
# 测试是否安装成功
import pymc3 as pm
# 输出版本号
print(pm.__version__)

### 收敛性

#### 自相关性

自相关性：用于衡量一串数字与自身的相关程度。其中 1 表示完美的正相关，0 表示完全无关， -1 表示 完美的负相关。

我们将序列 $x_t$ 在 $t$ 时刻与 $t-k$ 时刻的相关性，定义为：

$$R(k) = Corr( x_t, x_{t-k} ) $$

一种更容易理解自相关性的方式是，如果我拥有 $x_t$ 序列的前 $t - k$ 个数据，我能够很容易估计序列在 t 时刻的值，那么就可以说该序列在 $t$ 时刻 与 $t -k$ 时刻具有相关性。因此可以说，序列 $x$ 是自相关的。

那么，下面这两个式子是否是自相关的呢？

$$x_t \sim \text{Normal}(0,1), \;\; x_0 = 0$$
$$y_t \sim \text{Normal}(y_{t-1}, 1 ), \;\; y_0 = 0$$

首先序列 $x_t$ 不是自相关，它是由一些服从于正态分布的随机变量组成。因为，已知 $x_2$ 并不会有助于我们猜出 $x_3$ 是多少。

而从 $y_t$ 生成方式中就可以看出该序列是自相关。如果我们知道 $y_2 = 10$,那么 $y_3$ 的取值肯定距离 10 不远。同样，我们也可以猜到 $y_4$ 不大可能取 0 或 20，取 5 的可能性稍大。进一步，我们也可以猜测 $y_5$ 的范围。但是如果叫我们利用 $y_2$ 的值推测 $y_{100}$ 的范围时，就有点力不从心了。由此逻辑，我们可以得到以下结论：

随着两点间的 k（两点在序列上的时间间隔）越大，$y_t$ 在 $t$ 时刻的值 与 $t-k$ 时刻的值的自相关性越小。

 接下来让我们根据上面两个分布初始化一些数据，并对它们进行可视化： 

In [ ]:
import scipy.stats as stats
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

figsize(12.5, 4)
# 生成 x 序列
x_t = np.random.normal(0, 1, 200)
x_t[0] = 0

# 生成 y 序列
y_t = np.zeros(200)
for i in range(1, 200):
    y_t[i] = np.random.normal(y_t[i - 1], 1)

plt.plot(y_t, label="$y_t$", lw=3)
plt.plot(x_t, label="$x_t$", lw=3)
plt.xlabel("time, $t$")
plt.legend()

上图中，黄色序列代表 $x_t$，非自相关（又叫白色噪音）。蓝色序列代表 $y_t$ ，高度自相关。

当然一个序列的自相关性也是可以用具体的值来表示的。我们从 [相关网址](http://tinyurl.com/afz57c4) 中找到了求取自相关的函数如下：

In [ ]:
def autocorr(x):
        # 求取自行管
        result = np.correlate(x, x, mode='full')
        # 结果进行归一化
        result = result / np.max(result)
        # 由于correlate 本身是求取两个不同序列的相关系数
        # 因此，reslut 会存储： [第一个x与第二个x 的相关系数序列,第二个x与第一个x 的相关系数序列]
        # 因此这里我们result 序列的一半即可
        return result[result.size // 2:]


result = autocorr(x_t)
print("x 序列的长度：", len(x_t))
print("result 序列的长度：", len(result))

该函数输出的 reslut 序列中的第 $i$ 个数据表示序列 $x_t$ 在 $t$ 时刻 与 $t-i$ 时刻的相关系数。此时 $t=200$ 。

接下来，让我们利用该函数，得到 x 序列和 y 序列的自相关系数，并且进行可视化分析：

In [ ]:
colors = ["#348ABD", "#A60628", "#7A68A6"]
# 横坐标，表示第 i 条数据
x = np.arange(1, 200)
# 得到 y_t 序列的自相关序列
plt.bar(x, autocorr(y_t)[1:], width=1, label="$y_t$",
        edgecolor=colors[0], color=colors[0])
# 得到 x_t 序列的自相关序列
plt.bar(x, autocorr(x_t)[1:], width=1, label="$x_t$",
        color=colors[1], edgecolor=colors[1])

plt.legend(title="Autocorrelation")
plt.ylabel("measured correlation \nbetween $y_t$ and $y_{t-k}$.")
plt.xlabel("k (lag)")
plt.title("Autocorrelation plot of $y_t$ and $x_t$ for differing $k$ lags.")

从上图中可以看到，随着 $k$ 增加， $y_t$ 的自相关性从一个很大的值递减。相比之下，$x_t$ 的自相关性看起来就像噪音，因而推断 $x_t$ 序列是非自相关的。

上面我们已经了解了自相关性的概念与求解方法，那么这些与 MCMC 的收敛性有什么关系呢？

因为，MCMC 算法返回的采样结果是具有相关性的（因为我们采用的是“行走”的算法，总是从当前位置，移动到附件的某一个位置）。

如果一次采样过程的探索结果很好，那么表现出的自相关性也会很高。我们可以把采样的迹理解为一条蜿蜒缓慢的河流。如果我们知道河面上的那颗树叶此时的位置，那么我们就可以较为精确的估计它下一步会走到哪里。

反之，如果一个过程的自相关性很低，我们就可以把它称之为“高度融合”（对于高度融合，你无法知道每个元素的下一个位置）。

PyMC 的 Matplot 模块里有内置的自相关性的画图函数，该函数我们已经在上一个实验的末尾进行了介绍。

#### 稀释

但是，如果后验样本的相关性很高，又会引发另一个问题。因为很多的后处理算法都需要样本间的彼此独立。这个问题可以通过每隔 n 个样本，返回一个样本来解决或减轻样本间的自相关性。我们将这种方法称作稀释。从上面的图像，我们可以看出，随着间隔越来越大，它们的相关性会越来越好。

接下来，我们指定不同的间隔来对 序列 $y_t$ 进行取值。并观察，不同稀释稀释程度下，序列 $y_t$ 的自相关性。

In [ ]:
# 指定序列的个数
# 我们间隔分别 1， 2,3取三个长度为 max_x 的序列
max_x = 200 // 3 + 1
x = np.arange(1, max_x)

# 画出间隔1 的序列，即原序列。共 max_x 条数据
plt.bar(x, autocorr(y_t)[1:max_x], edgecolor=colors[0],
        label="no thinning", color=colors[0], width=1)

# 每间隔 2 取一个数据。共 max_x 条数据
plt.bar(x, autocorr(y_t[::2])[1:max_x], edgecolor=colors[1],
        label="keeping every 2nd sample", color=colors[1], width=1)
# 每间隔 3 取一个数据。共 max_x 条数据
plt.bar(x, autocorr(y_t[::3])[1:max_x], width=1, edgecolor=colors[2],
        label="keeping every 3rd sample", color=colors[2])

plt.autoscale(tight=True)
plt.legend(title="Autocorrelation plot for $y_t$", loc="lower left")
plt.ylabel("measured correlation \nbetween $y_t$ and $y_{t-k}$.")
plt.xlabel("k (lag)")
plt.title("Autocorrelation of $y_t$ (no thinning vs. thinning) \
at differing $k$ lags.")

如上图所示，增加稀释程度可以更快的减少相关性。但并不是说间隔越大越好，因为稀释程度的增加意味着需要进行更多的 MCMC 迭代才能保持同样的样本量。比如 10 000 个未稀释的样本相当于100 000 个以 10 为间隔进行稀释的样本（即使后者的相关性较小，更适合用于后面的处理，但是计算量太大）。

那么稀释程度以多少为宜呢？由于不论如何稀释，返回的样本都会存在一定的相关性，因此只需要稀释到自相关性快速趋近于 0 的位置即可。一般来说，最后确定的间隔不会超过 10。

### MCMC 的一些秘诀

如果，不是 MCMC 的计算复杂度不大，贝叶斯推断早就是一个应用非常广泛的方法了。可以说 MCMC 是导致很多用户放弃使用贝叶斯推断的主要原因。接下来，我们会提供一些启发式的方法来加速 MCMC 引擎并帮助它更快的收敛。

#### 较好的初始值

如果能够让 MCMC 算法以后验分布的附近位置为起点，那么就有利于算法在很短的时间内就能得到有效的采样。我们可以使用 value 参数来告诉算法，我们猜测的后验分布会在哪里。代码格式如下：

$$mu = pm.Uniform( "mu", 0, 100, testval = data.mean() )$$

当然，我们也可以用上一个试验提到的 `findMap` 来让计算机自己去估算一个较好的初始值，但是计算机估算的过程也需要一定的时间。因此提供一个用户自定义的初值是很有必要的。

#### 先验的选择

如果变量的先验分布选择的不好，那么 MCMC 算法可能无法收敛，或至少难以收敛。举个例子，如果我们选择的先验分布在某个点的概率为 0，那么很有可能，我们计算出来的后验分布在该点的概率也为 0 。 但是真实数据中的这个点却存在，那么这样的后验分布显然不适合真实数据。

因此，我们需要自己慎重的选择先验分布。通常，当我们发现我们的模型缺乏收敛性或者看不出样本在哪一个区域内比较集中的话，那么，这就暗示了先验的选择是有误的。这就是贝叶斯推断中常说的 **无名定理**。

### 实验总结

PyMC 为我们提供了贝爷推断的强大后台并且隐藏了内部实现机制。这样，可以很好的方便我们进行贝叶斯推断。本实验，主要讲解了 MCMC 算法的一些注意事项和秘诀。在下一个实验中，我们会介绍一种在统计学中非常重要的理论，大数定理。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>